In [1]:
#!/usr/bin/env python3.6
# -*- coding: utf-8 -*-

In [2]:
# Imports
import sys
import math
import numpy as np
from matplotlib import animation
from IPython.display import HTML
from matplotlib import pyplot as plt
plt.rcParams['animation.ffmpeg_path'] = '/usr/bin/ffmpeg'
import mpl_toolkits.mplot3d.axes3d as p3

In [3]:
np.random.seed(20)
np.set_printoptions(threshold=sys.maxsize)

In [4]:
%matplotlib inline

# Data

In [10]:
# Read data
path = '../../../data.csv'
motion = np.genfromtxt(path, delimiter=',', dtype=np.float64)
contacts = np.genfromtxt(path, delimiter=',', usecols=(46, 47), dtype=bool)
print("Motion data has a length of ", len(motion))

Motion data has a length of  19415


In [11]:
print(motion.shape)

(19415, 49)


# Dataset Preparation

In [12]:
import sklearn
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split

In [13]:
def yaw_from_quaternion(Q):
    """
    Covert a quaternion into a full three-dimensional rotation matrix.
 
    Input
    :param Q: A 4 element array representing the quaternion (q0,q1,q2,q3) 
 
    Output
    :return: A 3x3 element matrix representing the full 3D rotation matrix. 
             This rotation matrix converts a point in the local reference 
             frame to a point in the global reference frame.
    """
    # Extract the values from Q
    x = Q[0]
    y = Q[1]
    z = Q[2]
    w = Q[3]
     
    return np.arctan2(2 * (w*z + x*y), 1 - 2 * (y*y + z*z))

In [14]:
def quaternion_rotation_matrix(Q):
    """
    Covert a quaternion into a full three-dimensional rotation matrix.
 
    Input
    :param Q: A 4 element array representing the quaternion (q0,q1,q2,q3) 
 
    Output
    :return: A 3x3 element matrix representing the full 3D rotation matrix. 
             This rotation matrix converts a point in the local reference 
             frame to a point in the global reference frame.
    """
    # Extract the values from Q
    q0 = Q[3]
    q1 = Q[0]
    q2 = Q[1]
    q3 = Q[2]
     
    # First row of the rotation matrix
    r00 = 2 * (q0 * q0 + q1 * q1) - 1
    r01 = 2 * (q1 * q2 - q0 * q3)
    r02 = 2 * (q1 * q3 + q0 * q2)
     
    # Second row of the rotation matrix
    r10 = 2 * (q1 * q2 + q0 * q3)
    r11 = 2 * (q0 * q0 + q2 * q2) - 1
    r12 = 2 * (q2 * q3 - q0 * q1)
     
    # Third row of the rotation matrix
    r20 = 2 * (q1 * q3 - q0 * q2)
    r21 = 2 * (q2 * q3 + q0 * q1)
    r22 = 2 * (q0 * q0 + q3 * q3) - 1
     
    # 3x3 rotation matrix
    rot_matrix = np.array([[r00, r01, r02],
                           [r10, r11, r12],
                           [r20, r21, r22]])
                            
    return rot_matrix

In [15]:
def get_accelerations_footstep_dataset(dataset, footsteps, debug=False):
    idx = 1
    inputs = []
    labels = []
    
    while idx < len(footsteps):
        if idx > 0:
            # Compute time difference between footsteps
            time_difference = abs(dataset[footsteps[idx], 0] - dataset[footsteps[idx-1], 0])
            
            # Round velocity array
            dataset[footsteps[idx-1], 1:4] = np.round(dataset[footsteps[idx-1], 1:4], 1)
            dataset[footsteps[idx], 1:4] = np.round(dataset[footsteps[idx], 1:4], 1)
            
            # Skip if accelerating from 0.0 vel > 0.1
            #if dataset[footsteps[idx-1], 1] == 0.0 and dataset[footsteps[idx], 1] != 0.1:
            #    idx += 1
            #    continue
                
            # Skip if current velocity commanded is 0.0
            #if dataset[footsteps[idx], 1] == 0.0:
            #    idx += 1
            #    continue
                
            fl_rr_moving = contacts[footsteps[idx], 0]
            fr_rl_moving = contacts[footsteps[idx], 1]
                
            if fl_rr_moving == fr_rl_moving:
                print("Invalid footstep")
                idx += 1
                continue
                
            if time_difference < 0.4:
                fl_rr_moving = contacts[footsteps[idx], 0]
                fr_rl_moving = contacts[footsteps[idx], 1]
                
                if fl_rr_moving == fr_rl_moving:
                    idx += 1
                    continue
                    
                # Compute rotation matrix incurred
                R_curr = quaternion_rotation_matrix(dataset[footsteps[idx], 39:43])

                # Retrieve base poses in world frame
                prev_base = dataset[footsteps[idx-1], 16:19]
                curr_base = dataset[footsteps[idx], 16:19]

                # Compute world displacement
                world_displacement = curr_base - prev_base

                # Compute base displacement
                base_displacement = np.dot(R_curr.T, world_displacement)

                # Compute yaw displacement
                prev_yaw = yaw_from_quaternion(dataset[footsteps[idx-1], 39:43])
                curr_yaw = yaw_from_quaternion(dataset[footsteps[idx], 39:43])
                yaw = curr_yaw - prev_yaw
                
                if dataset[footsteps[idx-1], 1] > dataset[footsteps[idx], 1] and \
                   dataset[footsteps[idx], 19] > dataset[footsteps[idx-1], 19]:
                    idx += 2
                    continue

                inputs.append(dataset[footsteps[idx-1], 1:4].tolist() + # previous cmd
                              dataset[footsteps[idx], 1:4].tolist() + # current cmd
                              dataset[footsteps[idx-1], 19:23].tolist() + # current velocity
                              dataset[footsteps[idx-1], 4:16].tolist() + # feet position w.r.t CoM
                              [fl_rr_moving, fr_rl_moving]) # swiging booleans
                
                if fl_rr_moving:
                    # FL vectors
                    prev_fl = np.array([[dataset[footsteps[idx-1], 4], dataset[footsteps[idx-1], 5], 0]])
                    curr_fl = np.array([[dataset[footsteps[idx], 4], dataset[footsteps[idx], 5], 0]])

                    # RR vectors
                    prev_rr = np.array([[dataset[footsteps[idx-1], 13], dataset[footsteps[idx-1], 14], 0]])
                    curr_rr = np.array([[dataset[footsteps[idx], 13], dataset[footsteps[idx], 14], 0]])

                    # Feet displacements
                    fl_displacement = (curr_fl - prev_fl) + base_displacement
                    rr_displacement = (curr_rr - prev_rr) + base_displacement

                    labels.append([fl_displacement[0,0], fl_displacement[0,1], rr_displacement[0,0], rr_displacement[0,1]])
                else:
                    # FR vectors
                    prev_fr = np.array([[dataset[footsteps[idx-1], 7], dataset[footsteps[idx-1], 8], 0]])
                    curr_fr = np.array([[dataset[footsteps[idx], 7], dataset[footsteps[idx], 8], 0]])

                    # RL vectors
                    prev_rl = np.array([[dataset[footsteps[idx-1], 10], dataset[footsteps[idx-1], 11], 0]])
                    curr_rl = np.array([[dataset[footsteps[idx], 10], dataset[footsteps[idx], 11], 0]])

                    # Feet displacements
                    fr_displacement = (curr_fr - prev_fr) + base_displacement
                    rl_displacement = (curr_rl - prev_rl) + base_displacement

                    labels.append([fr_displacement[0,0], fr_displacement[0,1], rl_displacement[0,0], rl_displacement[0,1]])
            
        idx += 1
        
    # Convert to numpy array
    inputs_numpy = np.array(inputs)
    labels_numpy = np.array(labels)
    
    return inputs_numpy, labels_numpy

# Stack datasets
X_footsteps_accelerations_absolute, Y_footsteps_accelerations_absolute = get_accelerations_footstep_dataset(motion, np.array([x for x in range(len(motion))]))

# Log
print(X_footsteps_accelerations_absolute.shape)
print(Y_footsteps_accelerations_absolute.shape)

Invalid footstep
(18981, 24)
(18981, 4)


In [16]:
%store X_footsteps_accelerations_absolute
%store Y_footsteps_accelerations_absolute

Stored 'X_footsteps_accelerations_absolute' (ndarray)
Stored 'Y_footsteps_accelerations_absolute' (ndarray)


In [17]:
Y_footsteps_accelerations_absolute[1]

array([ 3.76987239e-02, -2.84286750e-03,  3.80168041e-02, -5.36085384e-05])

In [18]:
X_footsteps_accelerations_absolute[10001, :]

array([ 0.9       ,  0.        ,  0.        ,  0.9       ,  0.        ,
        0.        ,  0.85420865, -0.04655648, -0.10789797, -0.04906182,
        0.36250073,  0.13199221, -0.35409266,  0.12503703, -0.12949544,
       -0.36185032, -0.37495139,  0.12603348, -0.37211487, -0.13254079,
       -0.12870097, -0.36308116,  0.        ,  1.        ])

In [19]:
dic = dict()
for x in range(len(X_footsteps_accelerations_absolute)):
    key = str(round(X_footsteps_accelerations_absolute[x, 0], 1)) + str(round(X_footsteps_accelerations_absolute[x, 3], 1))
    if key in dic:
        dic[key][0].append(Y_footsteps_accelerations_absolute[x, 0])
        dic[key][1].append(X_footsteps_accelerations_absolute[x, 6])
        dic[key][2].append((X_footsteps_accelerations_absolute[x, 10],
                            X_footsteps_accelerations_absolute[x, 13]))
    else:
        dic[key] = [[Y_footsteps_accelerations_absolute[x, 0]], 
                    [X_footsteps_accelerations_absolute[x, 6]],
                    [(X_footsteps_accelerations_absolute[x, 10],
                      X_footsteps_accelerations_absolute[x, 13])]]


In [21]:
for key in dic.keys():
    #print(f"{key[:3]} -> {key[3:]}. Mean: {np.round(np.mean(dic[key][0]), 4)}. Std: {np.round(np.std(dic[key][0]), 3)}.")
    if float(key[:3]) == 1.1 and float(key[3:]) == 1.1:
        for x in range(len(dic[key][0])):
            print(f"Actual velocity: {dic[key][1][x]}. Displacement: {dic[key][0][x]}")

Actual velocity: 0.2338416576385498. Displacement: 0.318816767642622
Actual velocity: 0.9635299444198608. Displacement: 0.6914080231790708
Actual velocity: 1.2323169708251953. Displacement: 0.761771237303265
Actual velocity: 0.2954287528991699. Displacement: 0.35946590089918495
Actual velocity: 1.0075408220291138. Displacement: 0.6798418525446008
Actual velocity: 1.1892269849777222. Displacement: 0.7273297669028291
Actual velocity: 0.26867058873176575. Displacement: 0.3442026946521307
Actual velocity: 0.9574404358863831. Displacement: 0.6089372639764561
Actual velocity: 1.007550597190857. Displacement: 0.687587863882785
Actual velocity: 0.18625523149967194. Displacement: 0.28081897105828824
Actual velocity: 0.9181352257728577. Displacement: 0.6787593636224507
Actual velocity: 1.2026846408843994. Displacement: 0.7542983051374299
Actual velocity: 0.19381888210773468. Displacement: 0.3181837155121887
Actual velocity: 0.9899759292602539. Displacement: 0.6785316734589215
Actual velocity: 1.

In [22]:
#for key in dic.keys():
#    print(f"Prev: {key[:3]}. Actual: {key[3:]}. \n\nDisp: {dic[key]}\n\n.")

In [23]:
#for x in range(len(X_footsteps_accelerations_absolute)):
#    print(f"{X_footsteps_accelerations_absolute[x, 0]} -> {X_footsteps_accelerations_absolute[x, 3]}. Step: {np.round(Y_footsteps_accelerations_absolute[x, 0], 2)}. CoM: {np.round(X_footsteps_accelerations_absolute[x, -5:-4], 2)}. FL: {np.round(X_footsteps_accelerations_absolute[x, 10], 3)}. FR: {np.round(X_footsteps_accelerations_absolute[x, 13], 3)}. RL: {np.round(X_footsteps_accelerations_absolute[x, 16], 3)}. RR: {np.round(X_footsteps_accelerations_absolute[x, 19], 3)}")